# Greenblatt Magic formula

## The Magic Formula: Explanation and Formula

The **Magic Formula** is an investment strategy developed by **Joel Greenblatt** to identify high-quality companies that are also undervalued. It ranks companies based on two key financial ratios:

---

## 📌 Core Idea

> **Buy good companies at cheap prices.**

The strategy identifies:
- **"Good companies"** → those with high returns on capital (efficient use of capital).
- **"Cheap companies"** → those with high earnings yield (undervalued based on operating profits).

---

## 🧮 The original Formula

1. **Earnings Yield (EY):**
$$
\text{Earnings Yield} = \frac{\text{EBIT}}{\text{Enterprise Value}}
$$
- Measures how cheap the stock is.
- EBIT = Earnings Before Interest and Taxes
- Enterprise Value = Market Cap + Debt - Cash

2. **Return on Capital (ROC):**
$$
\text{Return on Capital} = \frac{\text{EBIT}}{\text{Net Working Capital} + \text{Net Fixed Assets}}
$$
- Measures the quality of the business (how efficiently it uses its capital).
### My filters
1. Add minimum revenue
2. Skip if net income is non-positive
3. Skip (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0)
4. Operating expenses must be non-negative
5. Exclude extreme accounting adjustments (per filteration in the code)
6. Remove absurdly high EBIT margins (e.g., > 200%)
---

### 🤖 LLM 10-K Analysis

We further refine the outcomes of the Magic Formula for **domestic** stocks using 10-K filings by analyzing them through an LLM pipeline. This step allows for an additional qualitative screen, identifying potential red flags or strategic strengths that may not be captured in the numerical ranking alone.


## 🔍 Implementation Steps

1. **Filter the universe**: Remove financials, utilities, and companies with very small market cap.
2. **Rank all remaining stocks** by:
   - Earnings Yield (high = better)
   - Return on Capital (high = better)
3. **Compute combined rank**:
   $$
   \text{Combined Rank} = \text{Rank}_{EY} + \text{Rank}_{ROC}
   $$
4. **Sort by Combined Rank** (lowest = best overall).
5. **Pick top N stocks** (e.g., top 20–30).
6. **Hold for 1 year**, rebalance annually.

---

## ✅ Why It Works

- Avoids paying too much for popular stocks.
- Focuses on operationally efficient, consistently profitable companies.
- Enforces a disciplined, rules-based approach.

---

## ⚠️ Notes and Caveats

- Avoids subjective judgement; however, **screening accuracy** depends on **quality of financial data**.
- May underperform in short-term or irrational markets.
- Works best over a multi-year horizon (3–5+ years).
- I have separated this into US and International stocks.
---


In [ ]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
import requests
import pandas as pd
from tqdm import tqdm

api_key = os.getenv('financial_modeling_prep_api_key')
assert api_key is not None

In [ ]:
# Retry settings: 5 attempts with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=10, max=60))
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_magic_formula(api_key, symbols, minimum_margin=0.4, minimum_marketcap=500_000_000, minimum_revenue=100_000_000):
    import pandas as pd
    from tqdm import tqdm

    base_url = "https://financialmodelingprep.com/stable"
    results = []

    for symbol in tqdm(symbols, desc="Processing symbols"):
        try:
            # Step 1: Check market cap
            ev_url = f"{base_url}/enterprise-values/?symbol={symbol}&apikey={api_key}"
            ev_data = fetch_json(ev_url)[0]
            market_cap = ev_data.get('marketCapitalization')  
            
            if not ev_data or market_cap < minimum_marketcap:
                continue

            # Step 2: Exclude financials and real estate sectors
            profile_url = f"{base_url}/profile?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            sector = profile_data[0].get('sector') 
            company_name = profile_data[0].get('companyName')
            
            if not profile_data or sector in ['Financial Services', 'Real Estate']:
                continue

            # Step 3: Fetch income statement
            income_url = f"{base_url}/income-statement/?symbol={symbol}&apikey={api_key}"
            income_data = fetch_json(income_url)[0]
            net_income = income_data.get('netIncome')
            revenue = income_data.get('revenue')

            # Filter: minimum revenue
            if revenue < minimum_revenue:
                continue

            # Skip if net income is non-positive (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0)
            if net_income is None or net_income <= 0:
                continue

            cash_url = f"{base_url}/cash-flow-statement/?symbol={symbol}&apikey={api_key}"
            cash_data = fetch_json(cash_url)[0]
            cashflow = cash_data['operatingCashFlow']
            
            #if abs(net_income - cashflow) > 0.5 * net_income:
            if (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0):
                print(f"Cashflow mismatch for {symbol}")
                continue  # net income not supported by cash flow

            # Step 4: Fetch balance sheet
            balance_url = f"{base_url}/balance-sheet-statement/?symbol={symbol}&apikey={api_key}"
            balance_data = fetch_json(balance_url)[0]

            # Step 5: Fetch dividend
            dividend = f"{base_url}/dividends/?symbol={symbol}&apikey={api_key}"
            dividend_data = fetch_json(dividend)
            dividend = 0
            if dividend_data:
                dividend_data = dividend_data[0]
                dividend = dividend_data.get('yield')
                div_date = dividend_data.get('date')

            # Extract relevant fields
            ebit = income_data.get('ebit')
            
            total_assets = balance_data.get('totalAssets')
            current_liabilities = balance_data.get('totalCurrentLiabilities')
            enterprise_value = ev_data.get('enterpriseValue')
            operating_expenses = income_data.get('operatingExpenses')
            other_expenses = income_data.get('otherExpenses')

            # Ensure all required fields are present
            if None in (ebit, revenue, total_assets, current_liabilities, enterprise_value):
                print(f"missing data for {symbol}")
                continue

            # Filter: Operating expenses must be non-negative
            if operating_expenses is not None and operating_expenses < 0:
                continue

            # Filter: exclude extreme accounting adjustments
            if other_expenses is not None and other_expenses < -revenue:
                continue

            # Filter: remove absurdly high EBIT margins (e.g., > 200%)
            ebit_margin = ebit / revenue
            if ebit_margin < minimum_margin or ebit_margin > 2:
                continue

            # Compute Greenblatt metrics
            earnings_yield = ebit / enterprise_value
            capital = total_assets - current_liabilities
            if capital <= 0:
                continue
            return_on_capital = ebit / capital

            # Append result
            results.append({
                'symbol': symbol,
                'company_name': company_name,
                'dividend': dividend,
                'div date': div_date,
                'sector' : sector,
                'ebit_margin': ebit_margin,
                'earnings_yield': earnings_yield,
                'return_on_capital': return_on_capital
            })

        except Exception as e:
            print(f'Symbol: {symbol}\n')
            print(e)

    # Step 6: Compile DataFrame and rank
    df = pd.DataFrame(results)
    if df.empty:
        return df

    df['ey_rank'] = df['earnings_yield'].rank(ascending=False)
    df['roc_rank'] = df['return_on_capital'].rank(ascending=False)
    df['combined_rank'] = df['ey_rank'] + df['roc_rank']
    df_sorted = df.sort_values(by='combined_rank')

    return df_sorted #[['symbol', 'ebit_margin', 'earnings_yield', 'return_on_capital', 'combined_rank']]


In [ ]:
symbols = pd.read_csv("russel1000.csv")
symbols = symbols['Ticker'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_marketcap=50_000_000, minimum_margin=0.05, minimum_revenue=0)

print(top_stocks)
russel1000_filtered = top_stocks
russel1000_filtered.head(30).to_csv('russel1000_magic_formula.csv')

In [ ]:
russel1000_filtered.head(30)

In [ ]:
symbols = pd.read_csv("all_listed_companies.csv")
symbols = symbols['Symbol'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.05)
print(top_stocks)
world_filtered = top_stocks
world_filtered[~world_filtered['symbol'].isin(russel1000_filtered['symbol'].to_list())].head(30).to_csv('world_magic_formula.csv')

## AI Driven Analysis on 10-K documents for domestic stock

In [ ]:
import sys
import pandas as pd
sys.path.append("../")
import importlib
import wallstreet_quant.edgar_pipeline  # or from wallstreet_quant import edgar_pipeline

importlib.reload(wallstreet_quant.edgar_pipeline)
from wallstreet_quant.edgar_pipeline import SecAnalysis
from wallstreet_quant.edgar_extractor import fetch_10K_and_10Q_filings

In [ ]:
russel1000_filtered = pd.read_csv('russel1000_magic_formula.csv')
sec_ai = SecAnalysis()

In [ ]:
from collections import defaultdict
filings = defaultdict(list)
for s in tqdm(russel1000_filtered['symbol']):
    try:
        filings[s] = fetch_10K_and_10Q_filings(s, "2023-01-01", "2025-6-6",form=["10-K"])
    except Exception as e:
        print(e)

In [ ]:
df = sec_ai(filings, model = "gpt-4o")  
df.to_excel('russel1000_magic_formula_gpt_analysis.xlsx')

In [ ]:
df